In [71]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Sep 27 15:01:51 2022

@author: thinhvietle
"""
import pandas
import numpy
import sklearn.impute as impute
import sklearn.preprocessing as preprocessing
import sklearn.model_selection as modelsel
import sklearn.tree as tree
import sklearn.metrics as metrics
import pydotplus
import collections
import sklearn
from sklearn.utils import resample

# LOAD DATA
dirname = '/Users/rkadhikari/Desktop/Advanced_Machine_Learning/'
#filename = 'Census_Supplement_sample.xlsx'
filename = 'events.csv'
df = pandas.read_csv(dirname + filename)
print(len(df))
df2= pandas.read_csv(dirname + filename)
df = df.drop(['id_odsp', 'id_event', 'sort_order', 'text', 'side', 'event_team', 
              'opponent','player','player2','player_in','player_out',"event_type","event_type2"], axis=1)
df = df.dropna()
df = df.drop_duplicates()

print(len(df))

L= [1,2,6,8,9,10]
O= [2,3]
df = df[~df.shot_place.isin(L)]
df = df[~df.shot_outcome.isin(O)]
print(len(df))

941009
69164
32403


In [72]:
indices2=df.index.values
df3=df2.loc[indices2,]

df3=df3.drop(['id_odsp', 'id_event', 'sort_order', 'text', 'side', 'event_team', 
              'opponent','player_in','player_out',"player2","event_type","event_type2"], axis=1)


In [73]:
rating_data=pandas.read_csv("/Users/rkadhikari/Desktop/Advanced_Machine_Learning/archive/players_16.csv")
rating_data=rating_data.loc[:,["short_name","overall","shooting"]]
rating_data=rating_data.dropna()

In [74]:
def name_coverter(check):
    check=check.lower()
    first=check.split()[0]
    last=check.split()[-1]
    name=first[0]+". "+last
    
    return(name)

import unidecode
def remove_accents(a):
    a=a.lower()
    return unidecode.unidecode(a)

In [75]:
df3["player"]=df3["player"].apply(remove_accents)
df3["player"]=df3["player"].apply(name_coverter)

In [76]:
rating_data['short_name'] =rating_data['short_name'].apply(remove_accents)
rating_data['short_name']=rating_data["short_name"].apply(name_coverter)
rating_data

,short_name,overall,shooting
0,l. messi,94,88.0
1,c. ronaldo,93,93.0
2,a. robben,90,86.0
4,l. suarez,90,88.0
5,e. hazard,89,82.0
...,...,...,...
15614,s. mcwilliams,44,41.0
15616,e. redman,44,19.0
15617,r. horne,44,40.0
15619,c. shephard,44,34.0


In [77]:
df3["overall"]=1
df3["shot_rating"]=1

In [78]:
unavailable_count=0
for i in range(len(df3["player"])):
    player_name=df3["player"].iloc[i]
    if (sum(rating_data["short_name"]==player_name)):
    
        index2=rating_data[rating_data["short_name"]==player_name].index.values[0]
        rating=rating_data.loc[index2,"overall"]
        shot_rating=rating_data.loc[index2,"shooting"]
        df3["overall"].iloc[i]=rating
        df3["shot_rating"].iloc[i]=shot_rating
    else:
        unavailable_count=unavailable_count+1

     

/var/folders/pn/16f1xmgx2h7_0r2pxpt6z09c0000gn/T/ipykernel_23919/2377797292.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3["overall"].iloc[i]=rating
/var/folders/pn/16f1xmgx2h7_0r2pxpt6z09c0000gn/T/ipykernel_23919/2377797292.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3["shot_rating"].iloc[i]=shot_rating


In [80]:
print(len(df3[df3["overall"]==1]["player"].unique()))
print(unavailable_count)

908
6388


## fill missing player rating with 2015 fifa data

In [81]:
check=[1]
missing_df=df3[df3.overall.isin(check)]
complete_df=df3[~df3.overall.isin(check)]

In [82]:
rating_data15=pandas.read_csv("/Users/rkadhikari/Desktop/Advanced_Machine_Learning/archive/players_15.csv")
rating_data15=rating_data15.loc[:,["short_name","overall","shooting"]]
rating_data15=rating_data15.dropna()

In [83]:
rating_data15['short_name'] =rating_data15['short_name'].apply(remove_accents)
rating_data15['short_name']=rating_data15["short_name"].apply(name_coverter)


In [84]:
unavailable_count=0
for i in range(len(missing_df["player"])):
    player_name=missing_df["player"].iloc[i]
    if (sum(rating_data15["short_name"]==player_name)):
    
        index2=rating_data15[rating_data15["short_name"]==player_name].index.values[0]
        rating=rating_data15.loc[index2,"overall"]
        shot_rating=rating_data15.loc[index2,"shooting"]
        missing_df["overall"].iloc[i]=rating
        missing_df["shot_rating"].iloc[i]=shot_rating
    else:
        unavailable_count=unavailable_count+1

/var/folders/pn/16f1xmgx2h7_0r2pxpt6z09c0000gn/T/ipykernel_23919/584657005.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_df["overall"].iloc[i]=rating
/var/folders/pn/16f1xmgx2h7_0r2pxpt6z09c0000gn/T/ipykernel_23919/584657005.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_df["shot_rating"].iloc[i]=shot_rating


In [85]:
complete2_df=missing_df[~missing_df.overall.isin(check)]
final_missing=missing_df[missing_df.overall.isin(check)]

In [86]:
final_complete_data=pandas.concat([complete_df,complete2_df],axis=0)

In [87]:
len(final_missing[final_missing["overall"]==1]["player"].unique())

601

In [88]:
manual_filled=pandas.read_csv("manual_rating.csv")
manual_filled=manual_filled.drop(['Unnamed: 0','Unnamed: 4'],axis=1)


manual_filled['player'] =manual_filled['player'].apply(remove_accents)
manual_filled['player']=manual_filled["player"].apply(name_coverter)


In [89]:
unavailable_count=0
for i in range(len(final_missing["player"])):
    player_name=final_missing["player"].iloc[i]
    if (sum(manual_filled["player"]==player_name)):
    
        index2=manual_filled[manual_filled["player"]==player_name].index.values[0]
        rating=manual_filled.loc[index2,"overall"]
        shot_rating=manual_filled.loc[index2,"shot_rating"]
        final_missing["overall"].iloc[i]=rating
        final_missing["shot_rating"].iloc[i]=shot_rating
    else:
        unavailable_count=unavailable_count+1

/var/folders/pn/16f1xmgx2h7_0r2pxpt6z09c0000gn/T/ipykernel_23919/2008811854.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_missing["overall"].iloc[i]=rating
/var/folders/pn/16f1xmgx2h7_0r2pxpt6z09c0000gn/T/ipykernel_23919/2008811854.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_missing["shot_rating"].iloc[i]=shot_rating


In [96]:
final2_complete_data=pandas.concat([final_complete_data,final_missing],axis=0)


In [97]:
median_overall=numpy.median(final2_complete_data["overall"])
median_shot_rating=numpy.median(final2_complete_data["shot_rating"])

In [98]:
final2_complete_data["overall"]=final2_complete_data["overall"].replace(1,median_overall)
final2_complete_data["shot_rating"]=final2_complete_data["shot_rating"].replace(1,median_shot_rating)


In [100]:
final2_complete_data

,time,player,shot_place,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break,overall,shot_rating
18,25,s. kagawa,7.0,4.0,0,3.0,1.0,1,1.0,0,82,67
22,26,m. hummels,5.0,1.0,0,3.0,3.0,2,1.0,0,86,58
26,29,m. gotze,5.0,1.0,1,3.0,1.0,1,1.0,0,84,73
92,80,r. tesche,3.0,1.0,1,3.0,2.0,0,3.0,0,71,67
147,24,m. ndjeng,3.0,1.0,0,9.0,1.0,1,1.0,0,70,62
...,...,...,...,...,...,...,...,...,...,...,...,...
938774,75,m. malcom,4.0,1.0,0,14.0,2.0,0,2.0,0,76,72
939407,91,s. niguez,5.0,1.0,0,3.0,1.0,0,1.0,0,71,39
940114,43,v. iborra,5.0,1.0,1,3.0,2.0,1,1.0,0,73,55
940139,63,v. iborra,5.0,1.0,1,19.0,1.0,0,1.0,0,73,55


In [101]:
final2_complete_data.to_excel("completed_data.xlsx")